In [3]:
# on vous donne la solution toute faite !
from pulp import *
import numpy as np
from gen_random import gen_random as gen_r
from prog_lineraire import prog_lineaire as prog_l

In [23]:
size_graph=20
size_cities=5
max_value=100
prob=0.2

random_graph,cities,graph_complet_cities = gen_r(size_graph,size_cities, max_value, prob)

In [74]:
print(random_graph)
print(cities)
print(graph_complet_cities)

[[ inf  97.  inf  inf  43.  inf  inf  inf  inf  inf  inf  inf  inf  34.
   inf  inf  98.  inf  inf  inf]
 [ 97.  inf  inf  79.  13.  inf  51.  82.  73.  inf  inf  inf  31.  inf
   54.  inf   4.  90.  54.  15.]
 [ inf  inf  inf  inf  inf  83.  41.  inf  92.  inf  77.   5.  inf  inf
   92.  inf 100.  47.  10.  inf]
 [ inf  79.  inf  inf  18.  22.  86.  inf  inf  85.  inf  inf  inf  inf
   inf  76.  80.  inf  inf  41.]
 [ 43.  13.  inf  18.  inf  inf  22.  inf  inf  28.  inf  inf  inf  inf
   75.  inf  49.  59.  28.  inf]
 [ inf  inf  83.  22.  inf  inf  inf  inf  inf  inf   2.  inf  inf  inf
   inf  inf  inf  inf  inf  inf]
 [ inf  51.  41.  86.  22.  inf  inf  73.  inf  inf  inf  18.  inf  inf
   inf  inf  inf  inf  30.  52.]
 [ inf  82.  inf  inf  inf  inf  73.  inf  91.  15.  inf  66.  inf  inf
   inf  inf  inf  79.  13.  21.]
 [ inf  73.  92.  inf  inf  inf  inf  91.  inf  inf  99.  inf  inf  inf
   63.  73.  inf  inf  inf  inf]
 [ inf  inf  inf  85.  28.  inf  inf  15.  inf  inf  10

In [98]:
import networkx as nx
import matplotlib.pyplot as plt

G = nx.from_numpy_matrix(graph_complet_cities) 
pos=nx.random_layout(G,[,20,20])
print(pos)

plt.figure(figsize=(20, 20))
for k, p in pos.items():
    plt.scatter(p[0], p[1], s=50, edgecolor='None')
plt.axis('equal')
plt.show()

SyntaxError: invalid syntax (3187503554.py, line 5)

In [72]:
route_plan,subtour,time,status,value = prog_l(graph_complet_cities)

print("Route :", route_plan)
print("Subtour :", subtour)
print("Time :", time)
print("Status :", status)
print("Value :", value)

Route : [[(0, 1), (1, 0)], [(2, 4), (4, 3), (3, 2)]]
Subtour : [2]
Time : 0.057546138763427734
Status : Optimal
Value : 151.0
